# Google Data Analytics Professional Certificate Capstone Case Study

## By JH Quek

In this capstone study, you are required to perform the five steps of data analysis: **ask, prepare, process, analyze, share, and act.** In the accompanying case study guide, you will find guiding questions to help you along, and the key deliverables you have to produce at the end of each step of the process.


---
## **ASK:**
---

<u>**Scenario**:</u> You are working as a Business Intelligence Analyst at a defense agency in the government. Recently, your agency is outsourcing an important project to an external defense contractor. Since will be an expensive undertaking, the project manager only wants the best defense company to be awarded the tender. Hence, he has tasked you to conduct an analysis of the historical data of the top 100 defense companies' revenue. Your job is to identify the most suitable firm(s) to be awarded the contract. At the last step of the analysis, you will present your results and give a recommendation to the project manager and chief finanicial officer (CFO) of the agency.

The metric of success for a company will be its gross revenue from defense contracting. 

---
## **PREPARE:**
---

The data that we use will be the <u>Defence Companies Top 100 for Each Year From 2005</u> dataset, obtained from the following Kaggle link: https://www.kaggle.com/datasets/onurduman/defence-companies-top-100-for-each-year-from-2005

This dataset is loaded into Excel and stored in the 'Data' folder of this working directory. This dataset was scrapped from Defense News, a global website focusing on defense and national security issues. The raw scrapped data was cleaned slightly, which we will further wrangle the data later. 

The metadata of the dataset is as follows:
<pre>
* Year : year
* Rank : position of the company relative to other within that year
* Company : name of the defense contractor
* Leadership : name of the CEO / President of the firm
* Country : country where the company is based
* Defense_Revenue_From_A_Year_Ago(in millions) : revenue from defense production 1 year ago
* Defense_Revenue_From_Two_Years_Ago(in millions) : revenue from defense production 2 years ago
* %Defense Revenue Change : percentage change of revenue from defense production 1 year ago
* Total Revenue(in millions) : total revenue generated 1 year ago
* %of Revenue from Defence : percentage of revenue from defense production between the last 2 years
</pre>

The datatset gives the revenue obtained from defense contracting 1 year ago. Nonetheless the ranking of the company in the current year is based on its perfromance 1 year ago. That is, for the current year for each company, the defense revenue last year is computed from its total revenue and its percentage of revenue from defense. Their ranking is sorted according to this value in the present year.


## **PROCESS:**

The first step is do an exploration of the dataset, to get an idea of what we can or cannot do with the data. I will be using the popular Pandas library in Python for the entiety of this case study - as this is what I use at work and what I am most familiar with. For data visualiztion, I will use Microsoft Power BI to create a dashboard in the Share phase of the analysis process.

* Check the data for errors
* Transform the data so you can work with it effectively
* Document the cleaning process

In [1]:
# import modules and dependencies and some display settings
import numpy as np
import pandas as pd
from datetime import datetime

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
np.set_printoptions(suppress = True)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Let us take an exploratory look at the entire dataset

In [2]:
path = 'H:/My Drive/Data Science and Analytics Portfolio/2 Projects/3 Google Data Analytics Capstone/2 Main/Data/'
#path = 'I:/My Drive/Data Science and Analytics Portfolio/2 Projects/3 Google Data Analytics Capstone/2 Main/Data/'
df = pd.read_excel(path + 'Raw/Defense Companies From 2005.xlsx')

df.head()

,Year,Rank,Company,Leadership,Country,Defense_Revenue_From_A_Year_Ago(in millions),Defense_Revenue_From_Two_Years_Ago(in millions),%Defense Revenue Change,Total Revenue(in millions),%of Revenue from Defence
0,2020,1,Lockheed Martin,"James D. Taiclet, President and CEO *",USA,56606.000,50536.000,12.011,59812.000,94.640
1,2020,2,Boeing,"David Calhoun, President and CEO *",USA,34300.000,34050.000,0.734,76559.000,44.802
2,2020,3,General Dynamics,"Phebe Novakovic, Chairman and CEO",USA,29512.000,27507.000,7.289,39350.000,74.999
3,2020,4,Northrop Grumman,"Kathy J. Warden, Chairman, CEO and President",USA,28600.000,25300.000,13.043,33841.000,84.513
4,2020,5,Raytheon Company,"Thomas Kennedy, Chairman and CEO",USA,27448.000,25163.940,9.077,29200.000,94.000


Let us print some basic statistics of the dataframe below

In [3]:
n = len(df)
unique_years = np.unique(df['Year'])
unique_countries = np.unique(df['Country'])
unique_companies = np.unique(df['Company'])
countries_counts = df['Country'].value_counts()
smallest_revenue_def = min(df['%of Revenue from Defence'])
largest_revenue_def = max(df['%of Revenue from Defence'])

print('Length of dataframe :', n, '\n')
print('Duration of years :', unique_years, '\n')
print('Countries of contractors :', unique_countries, '\n')
print('List of defense companies (first 10):', unique_companies[:10], '\n')
print('Top 5 countries where most defense contractors were based :\n', countries_counts[:5], '\n')
print('Highest and lowest revenue percentages from defense contracting :', largest_revenue_def, smallest_revenue_def)

Length of dataframe : 1604 

Duration of years : [2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018
 2019 2020] 

Countries of contractors : ['Australia' 'Belgium' 'Brazil' 'Canada' 'China' 'Finland' 'France'
 'Germany' 'India' 'Ireland' 'Israel' 'Italy' 'Japan' 'Kuwait'
 'Netherlands' 'Norway' 'Russia' 'Singapore' 'South Africa' 'South Korea'
 'Spain' 'Sweden' 'Switzerland' 'Turkey' 'USA' 'Ukraine' 'United Kingdom'] 

List of defense companies (first 10): ['AAI' 'AAR' 'AAR Corp' 'ADI' 'AECOM' 'ANA Holdings' 'ARINC'
 'ASELSAN Elektronik Sayi ve Ticaret A' 'ATK' 'Accenture'] 

Top 5 countries where most defense contractors were based :
 USA               683
United Kingdom    162
Russia            113
Japan             102
France             86
Name: Country, dtype: int64 

Highest and lowest revenue percentages from defense contracting : 120.90341 0.66690207


Immediately, we note that we have some problems:
* Complicated column names
* Duplicated company names - for example, Singapore Technologies Engineering and ST Engineering is the same company
* Spelling error in company names - Concern Radioelectronic Technologies JSC and Concern Rdaioelectronic Technologies JSC
* Very large range of percentage revenue from defense contracting  - from 0.667 to 121%
* The largest percentage from revenue is more than 100%
* The revenue for year 2013 is not scaled
* Profits in millions create too many digits in the revenue columns
* Confusing labelling of defense revenue - ranking of current year is based on last year

### Data Cleaning

Hence, in our data cleaning block, we will perform the following actions:
* Simplify column names
* Sort by the unique country names, we standardize the duplicated company names, and correct the mispelled one. This step involves a a good amount of googling! We'll stick to the latest company name as at 2022. These errors are very likely caused by changes in the company name
* Pay special attention to companies that sound similar but are actually separate entities
* Select only rows where the percentage of defense contracting revenue is more than 70%
* Scale the 2013 year revenue accordingly
* Scale the revenue columns to a billion
* We subtract 1 from the Year column, drop the Rank, Leadership and Defense_Revenue_From_A_Year_Ago(in millions) columns. This will **reflect the current year's defense revenue income**

In [4]:
# make a copy of the dataframe for us to work with
df_cleaned = df.copy()

# drop the three columns and reflect the current year's defense revenue
df_cleaned = df_cleaned.drop(columns = ['Rank', 'Leadership'], axis = 1).reset_index(drop = True)
df_cleaned['Year'] = df_cleaned['Year'].apply(lambda x : x - 1)

# we change the column names first
cols_renamed = {'Defense_Revenue_From_A_Year_Ago(in millions)' : 'Present Year Revenue',
                'Defense_Revenue_From_Two_Years_Ago(in millions)' : 'Last Year Revenue', 
                '%Defense Revenue Change' : 'Percentage Change', 
                'Total Revenue(in millions)' : 'Present Year Total Revenue', 
                '%of Revenue from Defence' : 'Percentage Defense Revenue'}

df_cleaned = df_cleaned.rename(columns = cols_renamed)

# next, we find print the unique company names for each of the countries
"""
for country in unique_countries:
    defense_company_of_country = df[df['Country'] == country]['Company']
    print(country, ':\n', np.unique(defense_company_of_country), '\n')
"""

# now we standardize the company names, with the essential help of Google
Finland_correction = {'Patria':'Patria Group', 
                      'Patria Oy':'Patria Group'}

France_correction = {'DCN':'Naval Group', 
                     'DCNS Group':'Naval Group', 
                     'Dassault':'Dassault Group', 
                     'GIAT Industries':'Nexter Systems', 
                     'Nexter':'Nexter Group', 
                     'Safran':'SAFRAN Group',
                     'SAGEM':'SAFRAN Group',
                     'Snecma':'SAFRAN Group'}

Germany_correction = {'Diehl Group':'Diehl Stiftung', 
                       'Rheinmetall':'Rheinmetall AG'}

India_correction = {'Bharat Electronics':'Bharat Electronics Limited', 
                    'Bharat Electronics Ltd':'Bharat Electronics Limited', 
                    'Hindustan Aeronautics':'Hindustan Aeronautics Limited', 
                    'Hindustan Aeronautics Ltd':'Hindustan Aeronautics Limited', 
                    'Hindustan Aeroutics':'Hindustan Aeronautics Limited'}

Israel_correction = {'Elbit Systems':'Elbit Systems Ltd', 
                     'Israel Aerospace Industries Ltd':'Israel Aerospace Industries', 
                     'Israel Aircraft Industries':'Israel Aircraft Industries', 
                     'Israel Military Industries':'IMI Systems', 
                     'Israel Military Industries Ltd':'IMI Systems', 
                     'Rafael Advanced Defense Systems':'Rafael Advanced Defense Systems Ltd'}

Italy_correction = {'Fincantieri':'Fincantieri SpA',
                    'Fincantieri  Cantieri vali Italiani SpA':'Fincantieri SpA',
                    'Finmeccanica':'Leonardo SpA', 
                    'Leonardo':'Leonardo SpA'}

Japan_correction = {'Ana Holdings':'ANA Holdings', 
                    'DSN':'DSN Corporation', 
                    'Fuji Heavy Industries':'Subaru Corporation', 
                    'Fujitsu':'Fujitsu Limited', 
                    'Fujitsu Ltd':'Fujitsu Limited', 
                    'IHI':'IHI Corporation', 
                    'IshikawajimaHarima Heavy Industries':'IHI Corporation', 
                    'Komatsu':'Komatsu Limited',
                    'Komatsu Ltd':'Komatsu Limited',
                    'Misubishi Heavy Industries':'Mitsubishi Heavy Industries Limited', 
                    'Mitsubishi Heavy Industries':'Mitsubishi Heavy Industries Limited', 
                    'Mitsubishi Heavy Industries Ltd':'Mitsubishi Heavy Industries Limited', 
                    'NEC':'NEC Corporation', 
                   'Universal Shipbuilding Corp':'Universal Shipbuilding Corporation'}

Netherlands_correction = {'Airbus':'Airbus SE', 
                          'Airbus Group':'Airbus SE', 
                          'EADS':'Airbus Defence and Space', 
                          'EADS NV':'Airbus Defence and Space'}

Norway_correction = {'Kongsberg':'Kongsberg Gruppen', 
                     'mmo':'Nammo'}

Russia_correction = {'Admiralteiske Verfi':'Admiralty Shipyards',
                     'Admiralteiskiye Verfi':'Admiralty Shipyards', 
                     'AlmazAntei':'Almaz-Antey',
                     'AlmazAntey':'Almaz-Antey', 
                     'Aviation Holding Co Sukhoi':'Sukhoi Aviation Holding Company', 
                     'Aviation Holding Company Sukhoi':'Sukhoi Aviation Holding Company', 
                     'Baltiisky Zavod':'Baltic Shipyard', 
                     'Concern RadioElectronic Technologies':'Concern Radio-Electronic Technologies', 
                     'Concern RadioElectronic Technologies JSC':'Concern Radio-Electronic Technologies', 
                     'Concern Radioelectronic Technologies JSC':'Concern Radio-Electronic Technologies', 
                     'Concern Rdaioelectronic Technologies JSC':'Concern Radio-Electronic Technologies',
                     'Irkut':'Irkut Corporation',
                     'KB Priborostroyeniya':'KBP Instrument Design Bureau',
                     'KB Priborostroyeniya Instrument Design Bureau':'KBP Instrument Design Bureau',
                     'MMPP Salyut':'Salyut Machine-Building Association',
                     'RSK MiG':'MiG', 'RTI':'STI Systems',
                     'RTI Sistema':'RTI Systems',
                     'RTI Systems Concern':'RTI Systems',
                     'Russias Helicopters':'Russian Helicopters',
                     'Tactical Missiles':'Tactical Missiles Corporation',
                     'Tactical Missiles Corporation JSC':'Tactical Missiles Corporation',
                     'Tula KB Priborostroyeniya':'KBP Instrument Design Bureau',
                     'Ufa Enginebuilding':'Ufa Engine Building Production Association',
                     'United Aircraft Corp':'United Aircraft Corporation',
                     'United Engine':'United Engine Corporation',
                     'United EngineBuilding':'United Engine-Building Corporation',
                     'United Enginebuilding':'United Engine-Building Corporation'}

Singapore_correction = {'Singapore Technologies Engineering':'ST Engineering', 
                        'Singapore Technologies Engineering Ltd':'ST Engineering',
                       'St Engineering':'ST Engineering'}

South_Korea_correction = {'Hanwha':'Hanwha Group', 
                          'Hyundai Rotem Company':'Hyundai Rotem', 
                          'Korea Aerospace Industries LTD': 'Korea Aerospace Industries', 
                          'Korea Aerospace Industries Ltd':'Korea Aerospace Industries', 
                          'LIG Nex':'LIG Nex1', 
                          'Rotem Co':'Hyundai Rotem', 
                          'Samsung Techwin':'Hanwha Techwin'}

Spain_correction = {'IZAR Construcciones Navales':'IZAR Construcctiones Navales', 
                   'Indra':'Indra Sistemas'}

Sweden_correction = {'Saab':'Saab AB', 
                     'Saab Group':'Saab AB', 
                     'Saab Ãke':'Saab AB'}

Switzerland_correction = {'RUAG':'RUAG Holding', 
                          'Ruag': 'RUAG Holding', 
                          'RUAG Suisse':'Ruag Suisse'}

Turkey_correction = {'ASELSAN Elektronik Sayi ve Ticaret A':'Aselsan', 
                     'Aselsan AS': 'Aselsan', 
                     'BMC Otomotiv Sanayi ve Ticaret AS':'BMC', 
                     'FNSS Savunma Sistemleri AS':'FNSS Defence Systems', 
                     'Havelsan AS':'HAVELSAN', 
                     'Roketsan AS':'Roketsan', 
                     'STM Savunma Teknolojileri Muhendislik ve Ticaret AS':'STM'}

USA_correction = {'AAI':'AAI Corporation', 
                  'AAR':'AAR Corp', 
                  'Aerojet Rocketdyne':'Aerojet Rocketdyne Holdings Inc',
                  'Allegheny Technologies Inc':'ATI Inc',
                  'Alliant Techsystems':'ATK',
                  'Anteon':'Anteon International',
                  'Ball': 'Ball Corporation',
                  'Ball Aerospace':'Ball Aerospace & Technologies Corp',
                  'Ball Aerospace  Technologies':'Ball Aerospace & Technologies Corp',
                  'Ball Corp': 'Ball Corporation',
                  'Bechtel':'Bechtel Group',
                  'CACI':'CACI International Inc',
                  'CACI International':'CACI International Inc',
                  'CACI Intertiol':'CACI International Inc',
                  'CSC':'Computer Sciences Corporation',
                  'Computer Sciences Corp':'Computer Sciences Corporation',
                  'CSRA':'CSRA Inc',
                  'Cubic':'Cubic Corporation',
                  'CurtisWright':'Curtiss-Wright Corporation',
                  'CurtissWright':'Curtiss-Wright Corporation',
                  'CurtissWright Corporation':'Curtiss-Wright Corporation',
                  'DCNS Group':'Naval Group',
                  'DRS Technologies':'Leonardo DRS',
                  'Day  Zimmerman':'Day & Zimmermann',
                  'Day  Zimmermann':'Day & Zimmermann',
                  'DynCorp International':'DynCorp',
                  'DynCorp Intertiol':'DynCorp',
                  'EDO':'EDO Corporation', 
                  'EDS':'Electronic Data Systems',
                  'Engility':'Engility Holdings Inc',
                  'Exelis':'Exelis Inc',
                  'FLIR':'Teledyne FLIR',
                  'Fluor':'Fluor Corporation',
                  'Force Protection': 'Force Protection Inc',
                  'General Electric':'GE',
                  'General Dymics':'General Dynamics Corporation',
                  'General Dynamics':'General Dynamics Corporation',
                  'Goodrich':'Goodrich Corporation',
                  'Griffon':'Griffon Corporation',
                  'Harris':'Harris Corporation',
                  'HewlettPackard':'Hewlett-Packard',
                  'Honeywell':'Honeywell International Inc', 
                  'Huntington Ingalls':'Huntington Ingalls Industries',
                  'ITT':'ITT Inc',
                  'ITT Exelis':'Exelis Inc',
                  'Jacobs':'Jacobs Solutions Inc',
                  'Jacobs Engineering':'Jacobs Engineering Inc',
                  'Jacobs Engineering Group':'Jacobs Engineering Inc',
                  'KBR':'KBR Inc',
                  'L Communications':'L-3 Communications Holdings',
                  'L Technologies':'L3 Technologies',
                  'LHarris Technologies':'L3Harris Technologies',
                  'Lockheed Martin':'Lockheed Martin Corporation',
                  'MC Dean':'MC Dean Inc',
                  'ManTech':'ManTech International Corporation',
                  'ManTech International':'ManTech International Corporation',
                  'ManTech Intertiol':'ManTech International Corporation',
                  'Mantech':'ManTech International Corporation',
                  'Mercury Systems':'Mercury Systems Inc',
                  'Moog':'Moog Inc',
                  'Northrop Grumman':'Northrop Grumman Corporation',
                  'Orbital Sciences':'Orbital Sciences Corporation',
                  'OshKosh Truck':'Oshkosh Corporation',
                  'Oshkosh':'Oshkosh Corporation',
                  'Oshkosh Truck':'Oshkosh Corporation',
                  'Peraton':'Peraton Inc',
                  'Perspecta':'Perspecta Inc',
                  'Raytheon':'Raytheon Company',
                  'SRA Intertiol':'SRA International',
                  'SRC':'SRC Inc',
                  'SRCSRCTec':'SRCTec',
                  'Science Applications International Corp':'SAIC',
                  'Sierra Nevada Corp':'Sierra Nevada Corporation',
                  'Stewart  Stevenson Services':'Stewart & Stevenson',
                  'Teledyne':'Teledyne Technologies',
                  'Telephonics':'Telephonics Corporation',
                  'Textron':'Textron Inc',
                  'Titan': 'Titan Corporation',
                  'TransDigm':'TransDigm Group',
                  'URS': 'URS Corporation',
                  'United Technologies':'United Technologies Corporation',
                  'United Technologies Corp':'United Technologies Corporation',
                  'United Techologies':'United Technologies Corporation',
                  'VSE':'VSE Corporation',
                  'ViaSat': 'Viasat Inc',
                  'Viasat': 'Viasat Inc',
                  'Wyle':'Wyle Laboratories',
                  'vistar':'ViStar Corporation'}

UK_correction = {'Alvis':'Alvis Car and Engineering Company Limited', 
                 'BAE':'BAE Systems', 
                 'Babcock International Group':'Babcock International', 
                 'Backbock International':'Babcock International',
                 'Babcock Intertiol':'Babcock International',
                 'Babcock Intertiol Group':'Babcock International',
                 'Chemring':'Chemring Group', 
                 'GKN':'GKN Group', 
                 'Meggitt ':'Parker Meggitt', 
                 'QinetiQ':'QinetiQ Group',
                 'QinetiQ Plc':'QinetiQ Group', 
                 'RollsRoyce':'Rolls Royce', 
                 'Ultra Electronic Holdings':'Ultra Electronics Holdings', 
                 'Ultra Electronics':'Ultra Electronics Holdings'}

company_corrections = Finland_correction | France_correction | Germany_correction     | India_correction       | \
                      Israel_correction  | Italy_correction  | Japan_correction       | Netherlands_correction | \
                      Norway_correction  | Russia_correction | Singapore_correction   | South_Korea_correction | \
                      Spain_correction   | Sweden_correction | Switzerland_correction | Turkey_correction      | \
                      USA_correction     | UK_correction 

df_cleaned.replace({'Company': company_corrections}, inplace = True) 

# scale the numbers by a million
df_cleaned['Present Year Revenue'] = df_cleaned['Present Year Revenue'].apply(lambda x : x / 1000000 if len(str(x)) >= 10 else x)
df_cleaned['Last Year Revenue'] = df_cleaned['Last Year Revenue'].apply(lambda x : x / 1000000 if len(str(x)) >= 10 else x)
df_cleaned['Present Year Total Revenue'] = df_cleaned['Present Year Total Revenue'].apply(lambda x : x / 1000000 if len(str(x)) >= 10 else x)

# scale all the revenue numbers by 1000
df_cleaned['Present Year Revenue'] = df_cleaned['Present Year Revenue'] /1000 
df_cleaned['Last Year Revenue'] = df_cleaned['Last Year Revenue'] / 1000 
df_cleaned['Present Year Total Revenue'] = df_cleaned['Present Year Total Revenue'] / 1000

# get rid of that one role with more than 100% revenue
df_cleaned = df_cleaned[df_cleaned['Percentage Defense Revenue'] <= 100]

df_cleaned.head()

,Year,Company,Country,Present Year Revenue,Last Year Revenue,Percentage Change,Present Year Total Revenue,Percentage Defense Revenue
0,2019,Lockheed Martin Corporation,USA,56.606,50.536,12.011,59.812,94.640
1,2019,Boeing,USA,34.300,34.050,0.734,76.559,44.802
2,2019,General Dynamics Corporation,USA,29.512,27.507,7.289,39.350,74.999
3,2019,Northrop Grumman Corporation,USA,28.600,25.300,13.043,33.841,84.513
4,2019,Raytheon Company,USA,27.448,25.164,9.077,29.200,94.000


#### Write the cleaned dataset to excel as output file - in anticipation for the Share phase of the analysis process

In [5]:
# create another column for date time formatting in anticipation of power bi dashboard
df_cleaned_copy = df_cleaned.copy()
df_cleaned_copy['Year Formatted'] = df_cleaned_copy['Year'].apply(lambda x : datetime.strptime(str(x), '%Y'))

df_cleaned_copy.to_excel(path + 'Cleaned/Defense Contractors_cleaned.xlsx', index = False)

## **ANALYZE:**
Now, we are ready to analyze the data. We will perform several aggregations on the cleaned dataframe to derive summary numbers for our analysis. We will perform some aggregation on the data so that it is useful and accessible. From the aggregated values, we can perform calculations to derive some summary statistics. Lastly, from the numbers and standing of the companies, we can identify trends and relationships.

#### Aggregate present year's defense revenue by country

In [6]:
df_cleaned.groupby(['Country']).agg('sum', numeric_only = True).sort_values(by = 'Present Year Revenue', ascending = False)[:10]

,Year,Present Year Revenue,Last Year Revenue,Percentage Change,Present Year Total Revenue,Percentage Defense Revenue
Country,,,,,,
USA,1349659,3667.431,3464.091,8130.157,9800.810,34496.823
United Kingdom,325852,650.669,642.785,852.273,1145.659,9429.086
France,172978,350.196,342.229,699.382,1015.139,5083.488
Russia,227174,224.281,196.687,1524.297,270.267,9291.931
Netherlands,34203,213.946,207.809,99.723,1038.780,434.298
China,32296,191.937,190.585,17.558,721.191,420.771
Italy,64368,181.103,172.756,270.591,352.587,1331.968
Israel,126703,112.663,106.457,411.331,130.282,5672.940
Japan,190989,110.915,96.723,940.007,2741.329,669.571


#### Aggregate every company and sum up their present year defense profits - present only the top 10 companies

In [7]:
df_grouped_companies = df_cleaned.groupby(['Company', 'Country']).agg('sum').sort_values(by = 'Present Year Revenue', ascending = False)
df_grouped_companies[:10]

,,Year,Present Year Revenue,Last Year Revenue,Percentage Change,Present Year Total Revenue,Percentage Defense Revenue
Company,Country,,,,,,
Lockheed Martin Corporation,USA,32184,667.512,636.856,88.439,720.544,1482.105
Boeing,USA,32184,483.573,462.361,100.320,1203.022,676.452
BAE Systems,United Kingdom,32184,406.902,405.675,23.088,447.961,1444.945
Northrop Grumman Corporation,USA,32184,369.524,354.050,104.381,457.378,1287.250
Raytheon Company,USA,32184,351.704,339.857,59.849,378.433,1486.659
General Dynamics Corporation,USA,32184,334.426,318.901,102.850,470.454,1145.754
Leonardo SpA,Italy,30169,154.634,146.981,134.074,268.067,877.239
United Technologies Corporation,USA,32184,151.125,142.744,156.860,883.735,274.119
Thales,France,32184,140.980,141.933,-0.213,269.928,847.327


#### Aggregate the top 10 companies and observe their performance by year

In [8]:
top_10_company_names = df_grouped_companies[:10].index.get_level_values('Company')
df_top_10 = df_cleaned[df_cleaned['Company'].isin(top_10_company_names)]

df_top_10.groupby(['Company', 'Year']).agg('sum', numeric_only = True)[:20]

Present Year Revenue  Last Year Revenue  Percentage Change  \
Company     Year                                                               
BAE Systems 2004                17.159             15.036             14.116   
            2005                20.345             17.159             18.566   
            2006                20.935             20.344              2.906   
            2007                25.071             20.935             19.753   
            2008                32.667             29.800              9.621   
            2009                33.419             32.709              2.171   
            2010                33.109             33.419             -0.926   
            2011                29.130             33.109            -12.019   
            2012                26.813             29.130             -7.954   
            2013                28.014             26.813              4.479   
            2014                25.449             28.014             -9.156   
            2015                25.278             25.449             -0.672   
            2016                23.622             25.278             -6.552   
            2017                22.380             23.622             -5.257   
            2018                22.477             22.380              0.435   
            2019                21.033             22.477             -6.425   
Boeing      2004                27.360             22.033             24.177   
            2005                30.464             27.360             11.345   
            2006                29.200             30.464             -4.149   
            2007                30.800             29.200              5.479   

                  Present Year Total Revenue  Percentage Defense Revenue  
Company     Year                                                          
BAE Systems 2004                      22.359                      76.742  
            2005                      25.431                      80.000  
            2006                      26.500                      79.000  
            2007                      26.968                      92.966  
            2008                      34.351                      95.098  
            2009                      35.094                      95.226  
            2010                      34.614                      95.655  
            2011                      30.723                      94.816  
            2012                      28.255                      94.896  
            2013                      29.802                      93.999  
            2014                      27.411                      92.841  
            2015                      27.357                      92.400  
            2016                      25.867                      91.320  
            2017                      25.288                      88.500  
            2018                      24.569                      91.487  
            2019                      23.370                      90.000  
Boeing      2004                      50.500                      54.178  
            2005                      52.457                      58.074  
            2006                      54.845                      53.241  
            2007                      61.530                      50.057

In [9]:
list(top_10_company_names)

['Lockheed Martin Corporation',
 'Boeing',
 'BAE Systems',
 'Northrop Grumman Corporation',
 'Raytheon Company',
 'General Dynamics Corporation',
 'Leonardo SpA',
 'United Technologies Corporation',
 'Thales',
 'L-3 Communications Holdings']

#### Aggregate the top 10 companies and calculate summary statistics - sort by the company's median present year revenue

In [10]:
df_top_10.groupby(['Company']).agg({'Present Year Revenue': ['sum', 'mean', 'median'], 
                                    'Percentage Change' : ['sum', 'mean', 'median'],
                                    'Percentage Defense Revenue':['sum', 'mean', 'median'], 
                                    }).sort_values(by = ('Present Year Revenue', 'median'), ascending = False)

Present Year Revenue                \
                                                 sum   mean median   
Company                                                              
Lockheed Martin Corporation                  667.512 41.719 41.311   
Boeing                                       483.573 30.223 30.750   
BAE Systems                                  406.902 25.431 25.174   
Raytheon Company                             351.704 21.982 22.306   
Northrop Grumman Corporation                 369.524 23.095 21.913   
General Dynamics Corporation                 334.426 20.902 19.642   
Leonardo SpA                                 154.634 10.309 10.219   
L-3 Communications Holdings                  119.566  9.964 10.163   
United Technologies Corporation              151.125  9.445  9.643   
Thales                                       140.980  8.811  8.696   

                                Percentage Change                \
                                              sum   mean median   
Company                                                           
Lockheed Martin Corporation                88.439  5.527  4.034   
Boeing                                    100.320  6.270  1.935   
BAE Systems                                23.088  1.443 -0.118   
Raytheon Company                           59.849  3.741  4.425   
Northrop Grumman Corporation              104.381  6.524  6.438   
General Dynamics Corporation              102.850  6.428  4.683   
Leonardo SpA                              134.074  8.938  3.873   
L-3 Communications Holdings                96.338  8.028  3.746   
United Technologies Corporation           156.860  9.804 10.712   
Thales                                     -0.213 -0.013 -0.922   

                                Percentage Defense Revenue                
                                                       sum   mean median  
Company                                                                   
Lockheed Martin Corporation                       1482.105 92.632 93.225  
Boeing                                             676.452 42.278 44.733  
BAE Systems                                       1444.945 90.309 92.620  
Raytheon Company                                  1486.659 92.916 93.000  
Northrop Grumman Corporation                      1287.250 80.453 80.066  
General Dynamics Corporation                      1145.754 71.610 75.960  
Leonardo SpA                                       877.239 58.483 56.600  
L-3 Communications Holdings                       1005.717 83.810 82.943  
United Technologies Corporation                    274.119 17.132 17.039  
Thales                                             847.327 52.958 50.732

#### Find the top 10 companies ranked by defense revenue for every year. Construct a dataframe and count the number of times the company appears in the top 10 ranking

In [11]:
df_copy = df_cleaned.copy()[['Year', 'Company', 'Present Year Revenue']]

df_yr = df_copy[df_copy['Year'] == 2004]
df_yr = df_yr.sort_values(by = 'Present Year Revenue', ascending = False).reset_index(drop = True)[:10]

for i in np.arange(2005, 2020):
    df_sort = df_copy[df_copy['Year'] == i]
    df_sort = df_sort.sort_values(by = 'Present Year Revenue', ascending = False).reset_index(drop = True)[:10]
    df_yr = pd.concat([df_yr, df_sort],  ignore_index = True, axis = 0)

    
count = []
unique_names = np.unique(df_yr['Company'])
for name in unique_names:
    name_count = df_yr[df_yr['Company'] == name]['Company'].count()
    count.append(name_count)
    
dict_hist = dict(zip(list(unique_names), count))
dict_hist

{'Airbus Defence and Space': 9,
 'Airbus SE': 9,
 'Almaz-Antey': 1,
 'Aviation Industry Corporation of China': 2,
 'BAE Systems': 16,
 'Boeing': 16,
 'China North Industries Group Corporation Limited': 2,
 'General Dynamics Corporation': 16,
 'Halliburton': 2,
 'Honeywell International Inc': 1,
 'L-3 Communications Holdings': 9,
 'L3 Technologies': 1,
 'L3Harris Technologies': 1,
 'Leonardo SpA': 10,
 'Leonardo Spa': 1,
 'Lockheed Martin Corporation': 16,
 'Northrop Grumman Corporation': 16,
 'Raytheon Company': 16,
 'Thales': 6,
 'United Technologies Corporation': 10}

#### Export the dataset to CSV as well

Create another field to indicate True or False if the company exists in the Top 10 Companies list. This is for the dashboard later

Bloodly hell this does not work!! Looker Studio does not recognize the boolean variable.

In [12]:
df_cleaned_export = df_cleaned_copy.copy()
#df_cleaned_export['TOP 10'] = df_cleaned_export['Company'].apply(lambda x: True if x in list(top_10_company_names) else False)

df_cleaned_export.head(2)

,Year,Company,Country,Present Year Revenue,Last Year Revenue,Percentage Change,Present Year Total Revenue,Percentage Defense Revenue,Year Formatted
0,2019,Lockheed Martin Corporation,USA,56.606,50.536,12.011,59.812,94.640,2019-01-01
1,2019,Boeing,USA,34.300,34.050,0.734,76.559,44.802,2019-01-01


In [13]:
%%time
df_cleaned_export.to_csv(path + 'Cleaned/Defense Contractors_cleaned.csv', index = False)

CPU times: total: 15.6 ms
Wall time: 88.1 ms


#### Summary of Analysis:
Using the cleaned dataframe from the previous step, we perform the following actions:
* Aggregate present year's defense revenue by country
* Aggregate every company and sum up their present year defense profits - present only the top 10 companies
* Aggregate the top 10 companies and observe their performance by year
* Aggregate the top 10 companies and calculate summary statistics - sort by the company's median present year revenue
* Find how many times a company appear in top 10 revenue from 2004 to 2019

Now, we are ready to move onto the second last step of the analysis process: Share.

## **SHARE:**

We use Power BI to create a dashboard called Defense Contracting Market Dashboard. The dashboard has two spages - the first shows an overview of the entire defense contracting market. The second page limits the visualizations to only the top 10 companies, based on our analysis in the previous step. Using the dashboard, we can change various settings and filters on the data visualizations. From the settings, we can summarize our findings as follows:

The top five countries with the largest defense contracting market are: 

1. USA 
2. UK
3. France 
4. Russia
5. Netherlands

with a combined market share of <u>**5.11 trillion USD**</u>, over 2004 to 2019.

The top performer in terms of yearly defense revenue is <u>**Lockheed Martin Corporation**</u>, with a mean yearly defense profit of <u>**41.72 billion USD**</u>.

The top five defense companies - ranked in terms of median yearly defense revenue - are:

1. Lockheed Martin Corporation 
2. Boeing
3. BAE Systems
4. Raytheon Company
5. Northrop Grumman Corporation

with a combined yearly defense revenue of <u>**2.28 trillion USD**</u> over the period 2004 to 2019. With the exception of BAE Systems - which is British - the rest of the four companies are American. These companies are consistently ranked as top five most profitable companies during this period.

If we look at the growth for these five companies, the one with the most steady growth is <u>**Raytheon Company**</u>. This can be seen from the Growth against Year graph. Raytheon displays the least fluctuation compared to the rest.

## **ACT:**

From our findings in the previous steps, I present the following recommendations:
* Lockheed Martin Corporation should be awarded the project tender. With a median yearly defense revenue of 41.72 billion USD, it is the top defense firm. Furthermore, by working with Lockheed Martin, we have access to the lucrative US defense contracting market - with a total market valuation of 3.67 trilion USD, it represents the world's largest share of contractors.
* Besides Lockheed Martin, other top five companies such as Boeing, Raytheon and Northrop Grumman can also be taken into consideration. These companies also allow us to gain access to the US market.
* Now that we have identified the best firm to be awarded the tender, we can ask further specific questions such as:
   1. Which division are we working with? 
   2. How many key deliverables are there in our project? 
   3. Do we work with more than one division in Lockheed Martin?
* Lastly, we can dig deeper into accesing the financial performance of the company by looking at various financial indicators, such as their cash flow and income statements.